In [ ]:
import os
os.getcwd()

In [ ]:
path= "/content/drive/MyDrive/ESAA/Santander_Customer_Satisfaction"
os.chdir(path)
os.getcwd()

#분류 실습-캐글 산탄데르 고객 만족 예측

캐글의 산탄데르 고객 만족 데이터 세트에 대해서 고객 만족 여부를 XGBoost 와 LightGBM을 활용해 예측해보자. 370개의 피처로 주어진 데이터 세트 기반에서 고객 만족 여부를 예측하는 것이다. 클래스 레이블 명은 TARGET이며, 이 값이 1이면 불만을 가진 고객, 0이면 만족한 고객이다. 모델의 성능 평가는 ROC-AUC(ROC 곡선 영역)로 평가합니다. 대부분이 만족이고 불만족인 데이터는 일부일 것이기 때문에 정확도 수치보다는 ROC-AUC가 더 적합하다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

cust_df = pd.read_csv("./train_santander.csv",encoding='latin-1')
print('dataset shape:',cust_df.shape)
cust_df.head(3)


클래스 값 칼럼을 포함한 피처가 371개 존재한다. <br>

In [ ]:
cust_df.info()

111개의 피처가 float형, 260개의 피처가 int형으로 모든 피처가 숫자형이며, Null 값은 없다. 전체 데이터에서 만족과 불만족의 비율을 살펴보자.

In [ ]:
print(cust_df['TARGET'].value_counts())

In [ ]:
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()   
unsatisfied_cnt     #레이블인 TARGET == 1 인 값들의 개수 : 3008개

In [ ]:
total_cnt = cust_df.TARGET.count()
total_cnt    #TARGET 레이블의 총 개수 : 76020개

In [ ]:
print('unsatisfied 비율 {0:.2f}'.format((unsatisfied_cnt/total_cnt)))

대부분이 만족이며 불만족인 고객은 얼마 되지 않는 4%에 불과한다.

In [ ]:
#각 피처의 값 분포를 간단히 확인 해보자.
cust_df.describe()

In [ ]:
print(cust_df.var3.value_counts()[:10])   #-999999가 116개 존재함.

var3칼럼의 경우 min 값이 -999999 입니다. NaN이나 특정 예외 값을 -999999로 변환했을 것이다. var3은 숫자형이고, 다른 값에 비해 -999999 는 너무 편차가 심하므로 -999999를 가장 값이 많은 2로 변환하겠다. ID 피처는 단순 식별자에 불과하므로 피처를 드롭하겠다. 그리고 클래스 데이터 세트와 피처 데이터 세트를 분리해 별도의 데이터 세트로 별도로 저장하겠다.

In [ ]:
cust_df['var3'].replace(-999999,2,inplace=True)
cust_df.drop('ID', axis=1,inplace=True)

#피처 세트와 레이블 세트 분리, 레이블 칼럼은 DataFrame의 맨 마지막에 위치해 칼럼 위치 -1로 분리
X_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]
print('피처 데이터 shape:{0}'.format(X_features.shape))

In [ ]:
cust_df

학습과 성능 평가를 위해서 원본 데이터 세트에서 학습 데이터 세트와 테스트 데이터 세트를 분리하겠습니다. 비대칭한 데이터 세트이므로 TARGET 값 분포도가 학습 데이터와 테스트 데이터 세트에 모두 비슷하게 추출됐는지 확인하겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_features,y_labels,test_size=0.2,random_state=0)

train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape,X_test.shape))
print(train_cnt)
print(test_cnt)

In [ ]:
print('학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습과 테스트 데이터 세트 모두 TARGET의 값의 분포가 원본 데이터와 유사하게 전체 데이터의 4% 정도의 불만족 값(값1)으로 만들어졌다.

## XGBoost 모델 학습과 하이퍼 파라미터 튜닝

사이킷런 래퍼인 XGBClassifier를 기반으로 학습을 수행한다. 성능 평가 기준이 ROC-AUC 이므로 XGBClassifier가 eval_metric은 'auc'로 하겠다.(logloss로 해도 큰 차이는 없다.) 그리고 평가 데이터 세트는 앞에서 분리한 테스트 데이터 세트를 이용하겠다. 사실 테스트 데이터 세트를 XGBoost의 평가 데이터 세트로 사용하면 과적합의 가능성을 증가시킬 수 있지만, 여기서는 이러한 점만 주지하고 넘어가도록 하겠다.

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

#n_estimators는 500으로, random_state는 예제 수행 시마다 동일 예측 결과를 위해 설정
xgb_clf = XGBClassifier(n_estimators=500,random_state=156)

#성능 평가 지표를 auc로, 조기 중단 파라미터는 100으로 설정하고 학습 수행
xgb_clf.fit(X_train,y_train,early_stopping_rounds=100,eval_metric='auc',
            eval_set=[(X_train,y_train),(X_test,y_test)])
xgb_roc_score = roc_auc_score(y_test,xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC:{0:.4f}'.format(xgb_roc_score))

테스트 데이터 세트로 예측 시 ROC AUC는 약 0.8419이다.

XGBoost의 하이퍼 파라미터 튜닝<br>
칼럼의 개수가 많으므로 과적합 가능성을 가정하고, 3개 하이퍼 파라미터만 일차 튜닝 대상으로 하겠다. GridSearchCV에서 제학습된 estimator에서 ROC-AUC 수치가 어떻게 향상됐는지 확인해보자

In [ ]:
from sklearn.model_selection import GridSearchCV

#하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators 를 100으로 감소
xgb_clf = XGBClassifier(n_estimators=100)

params = {'max_depth':[5,7],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75]}

#cv는 3으로 가정
gridcv = GridSearchCV(xgb_clf,param_grid=params,cv=3)
gridcv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric='auc',
           eval_set=[(X_train,y_train),(X_test,y_test)])
print('GridSearchCV 최적 파라미터:',gridcv.best_params_)

xgb_roc_score = roc_auc_score(y_test,gridcv.predict_proba(X_test)[:,1],
                              average='macro')
print('ROC AUC:{0:.4f}'.format(xgb_roc_score))

ROC AUC 가 약 0.8461로 이전 테스트보다 살짝 향상된 결과를 보인다.

In [ ]:
#각 피처의 중요도를 구해보자.
from xgboost import plot_importance
import matplotlib.pyplot as plt 
%matplotlib inline

fig,ax = plt.subplots(1,1,figsize=(10,8))
plot_importance(xgb_clf,ax=ax,max_num_features=20,height=0.4)

XGBoost의 예측 성능을 좌우하는 가장 중요한 피처는 var38,var15 순입니다. 

##LightGBM 모델 학습과 하이퍼 파라미터 튜닝

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=500)

evals=[(X_test,y_test)]
lgbm_clf.fit(X_train,y_train,early_stopping_rounds=100,eval_metric='auc',
             eval_set=evals,verbose=True)
lgbm_roc_score = roc_auc_score(y_test,lgbm_clf.predict_proba(X_test)[:,1],
                               average='macro')
print('ROC AUC :{0:.4f}'.format(lgbm_roc_score))

LightGBM 수행 결과 ROC AUC가 약 0.8396을 나타낸다.

In [ ]:
from sklearn.model_selection import GridSearchCV

#하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 200으로 감소
lgbm_clf = LGBMClassifier(n_estimators=200)

params={'num_leaves':[32,64],'max_depth':[128,160],
        'min_child_samples':[60,100],'subsample':[0.8,1]}

#cv는 3으로 가정
gridcv = GridSearchCV(lgbm_clf,param_grid=params,cv=3)
gridcv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric='auc',
           eval_set=[(X_train,y_train),(X_test,y_test)])
print('GridSearchCV 최적 파라미터:',gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test,gridcv.predict_proba(X_test)[:,1],
                               average='macro')
print('ROC AUC:{0:.4f}'.format(lgbm_roc_score))

GridSearchCV 최적 파라미터: {'max_depth': 128, 'min_child_samples': 100, 'num_leaves': 32, 'subsample': 0.8}  
ROC AUC:0.8442

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators=100,num_leaves=32,subsample=0.8,
                          min_child_samples=100,max_depth=128)
evals=[(X_test,y_test)]
lgbm_clf.fit(X_train,y_train,early_stopping_rounds=100,eval_metric='auc',
             eval_set=evals,verbose=True)
lgbm_roc_score=roc_auc_score(y_test,lgbm_clf.predict_proba(X_test)[:,1],
                             average='macro')
print('ROC AUC:{0:.4f}'.format(lgbm_roc_score))

LightGBM의 경우 테스트 데이터 세트에서 ROC-AUC가 약 0.8442로 측정되었다.

#분류 실습-캐글 신용카드 사기 검출

##오버샘플링  
SMOTE 방법  
적은 데이터 세트에 있는 개별 데이터들의 k 최근접 이웃(k Nearest Neighbor)을 찾아서 이 데이터와 k개 이웃들의 차이를 일정 값으로 만들어서 기존 데이터와 약간 차이가 나는 새로운 데이터들을 생성하는 방식

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

card_df = pd.read_csv('/content/drive/MyDrive/ESAA/Credit_Card_Fraud_Detection/creditcard.csv')
card_df.head(3)


Time 피처의 경우는 데이터 생성 관련한 작업용 속성으로서 큰 의미가 없기에 제거한다.  Amount 피처는 신용카드 트랙잭션 금액 / Class는 레이블로서 0의 경우 정상, 1의 경우 사기 트랜잭션이다.

In [ ]:
card_df.info()

전체 284,807개의 레코드에서 결측치 값은 없으며, Class 레이블만 int형이고 나머지 피처들은 모두 float형이다.

In [ ]:
from sklearn.model_selection import train_test_split

#인자로 입력받은 DataFrame을 복사한 뒤 Time 칼럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop('Time',axis=1,inplace=True)
    return df_copy

In [ ]:
#사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수
def get_train_test_dataset(df=None):
    #인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    #DataFrame의 맨 마지막 칼럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:,:-1]
    y_target = df_copy.iloc[:,-1]
    #train_test_split()으로 학습과 테스트 데이터 분할. stratify=y_target 으로 Stratified 기반 분할
    X_train,X_test,y_train,y_test = \
    train_test_split(X_features,y_target,test_size=0.3,random_state=0,stratify=y_target)
    #학습과 테스트 데이터 세트 반환
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test = get_train_test_dataset(card_df)

In [ ]:
#생성한 학습 데이터 세트와 테스트 데이터 세트의 레이블 값 비율 확인해보자
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0]*100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0]*100)

0.172% , 0.173%로 큰 차이가 없이 잘 분할 되었다.

## 로지스틱 회귀를 이용해 신용 카드 사기 여부를 예측해보자

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
def get_clf_eval(y_test,pred=None,pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 =f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred_proba)
    print("오차 행렬")
    print(confusion)
    print("정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC={4:.4f}".format(accuracy,precision,recall,f1,roc_auc))

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]

#3장에서 사용한 get_clf_eval()함수를 이용해 평가 수행
get_clf_eval(y_test,lr_pred,lr_pred_proba)

In [ ]:
#인자로 사이킷런의 Estimator객체와 학습/테스트 세트를 입력 받아서 학습/예측/평가 수행.
def get_model_train_eval(model,ftr_train=None,ftr_test=None,tgt_train=None,tgt_test=None):
    model.fit(ftr_train,tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:,1]
    get_clf_eval(tgt_test,pred,pred_proba)

본, 데이터 세트는 극도로 불균형한 레이블 값 분포도를 가지고 있으므로 LGBMClassifier 객체 생성 시 boost_from_average = False 로 파라미터를 설정해야 한다.

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=1,
                          boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=X_train,ftr_test=X_test,
                     tgt_train=y_train,tgt_test=y_test)

LightGBM으로 모델을 학습해보면,     재현율 0.7568, ROC-AUC 0.9797로 앞의 로지스틱 회귀보다는 높은 수치를 나타낸다.

##데이터 분포도 변환 후 모델 학습/예측/평가

로지스틱 회귀는 선형 모델이다. 대부분의 선형 모델은 중요 피처들의 값이 정규 분포 형태를 유지하는 것을 선호한다.  

In [ ]:
#Amount 피처의 분포도를 확인해보자
import seaborn as sns
plt.figure(figsize=(8,4))
plt.xticks(range(0,30000,1000),rotation=60)
sns.distplot(card_df['Amount'])

카드 사용금액이 1000불 이하인 데이터가 대부분이며, 27,000불까지 드물지만 많은 금액을 사용한 경우가 발생하면서 꼬리가 긴 형태의 분포 곡선을 가지고 있다.Amount를 표준 정규 분포 형태로 변환한 뒤에 로지스틱 회귀의 예측 성능을 측정해보자

In [ ]:
from sklearn.preprocessing import StandardScaler
#사이킷런의 StandardScaler를 이용해 정규 분포 형태로 Amount 피처값 변환하는 로직으로 수정.
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    scaler = StandardScaler()
    amount_n = scaler.fit_transform(df_copy['Amount'].values.reshape(-1,1))
    #변환된 Amount를 Amount_Scaled로 피처명 변경후 DataFrame 맨 앞 칼럼으로 입력
    df_copy.insert(0,'Amount_Scaled',amount_n)
    #기존 Time,Amount 피처 삭제
    df_copy.drop(['Time','Amount'],axis=1,inplace=True)
    return df_copy

In [ ]:
X_train,X_test,y_train,y_test = get_train_test_dataset(card_df)

print('### 로지스틱 회귀 예측 성능 ###')
get_model_train_eval(lr_clf,ftr_train=X_train,ftr_test=X_test,
                     tgt_train=y_train,tgt_test=y_test)
print('### LightGBM 예측 성능 ###')
get_model_train_eval(lgbm_clf,ftr_train=X_train,ftr_test=X_test,
                     tgt_train=y_train,tgt_test=y_test)

두 모델 모두 정밀도, 재현율, ROC-AUC 에서 약간씩 성능이 개선되었음을 나타내고 있다.

## 이상치 데이터 제거 후 모델 학습/예측/평가

이상치 데이터를 IGR을 이용해 제거해 보겠다. 어떤 피처의 이상치 데이터를 검출할 것인지 선택이 필요하다. 결정값(레이블)과 가장 상관성이 높은 피처들을 위주로 이상치를 검출하는 것이 좋다.

In [ ]:
#corr()을 이용해 각 피처별로 상관도를 구한 뒤 시본의 heatmap을 통해 시각화해보자
import seaborn as sns
plt.figure(figsize=(9,9))
corr = card_df.corr()
sns.heatmap(corr,cmap='RdBu')

양의 상관관계가 높을수록 색깔이 진한 파란색에 가까우며, 음의 상관관계가 높을수록 색깔이 진한 빨간색에 가깝게 표현된다. Class 피처와 음의 상관관계가 가장 높은 피처는 V14와 V17이다. 이중 V14에 대해서만 이상치를 찾아서 제거해보자

In [ ]:
import numpy as np

def get_outlier(df=None,column =None,weight=1.5):
    #fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함.
    fraud = df[df['Class']==1][column]
    quantile_25 = np.percentile(fraud.values,25)
    quantile_75 = np.percentile(fraud.values,75)
    #IQR을 구하고, IQR에 1.5를 곱해 최댓값과 최솟값 지점 구함.
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr*weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    #최댓값보다 크거나, 최솟값보다 작은 값을 이상치 데이터로 설정하고 DataFrame index 반환.
    outlier_index = fraud[(fraud<lowest_val)|(fraud>highest_val)].index
    return outlier_index

In [ ]:
outlier_index = get_outlier(df=card_df,column='V14',weight=1.5)
print('이상치 데이터 인덱스:',outlier_index)

이상치 데이터 인덱스: Int64Index([8296, 8615, 9035, 9252], dtype='int64')   총 4개의 데이터가 이상치로 추출되었다.

In [ ]:
#get_processed_df()를 로그 변환 후 V14 피처의 이상치 데이터를 삭제하는 로직으로 변경.
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    amount_n = np.log1p(df_copy['Amount'])
    df_copy.insert(0,'Amount_Scaled',amount_n)
    df_copy.drop(['Time','Amount'],axis=1,inplace=True)
    #이상치 데이터 삭제하는 로직 추가
    outliear_index = get_outlier(df=df_copy,column='V14',weight=1.5)
    df_copy.drop(outlier_index,axis=0,inplace=True)
    return df_copy

X_train,X_test,y_train,y_test = get_train_test_dataset(card_df)

print('### 로지스틱 회귀 예측 성능 ###')
get_model_train_eval(lr_clf,ftr_train=X_train,ftr_test=X_test,
                     tgt_train=y_train,tgt_test=y_test)
print('### LightGBM 예측 성능 ###')
get_model_train_eval(lgbm_clf,ftr_train=X_train,ftr_test=X_test,
                     tgt_train=y_train,tgt_test=y_test)

이상치 데이터를 제거한 뒤, 로지스틱 회귀와 LightGBM 모두 예측 성능이 크게 향상되었다. 로지스틱 회귀의 경우 재현율이 60.81%에서 67.12%로 크게 증가했으며, LightGBM의 경우도 76.35%에서 82.88%로 역시 크게 증가했다.

## SMOTE 오버 샘플링 적용 후 모델 학습/예측/평가

SMOTE를 적용할 때는 반드시 학습 데이터 세트만 오버 샘플링을 해야 한다.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_sample(X_train,y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:',X_train.shape,y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트:',X_train_over.shape,y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포:\n',pd.Series(y_train_over).value_counts())